In [66]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit import Parameter, ParameterVector
from susy_qm import calculate_Hamiltonian, ansatze

import numpy as np

In [75]:
potential= "QHO"
cutoff = 2


H = calculate_Hamiltonian(cutoff, potential)
num_qubits = int(1 + np.log2(cutoff))
observable = SparsePauliOp.from_operator(H)

num_copies = 3


In [76]:
def _tensor_power(op, k):
    """Return op^(⊗k); for k=0 return None."""
    if k == 0:
        return None
    out = op
    for _ in range(k - 1):
        out = out.tensor(op)
    return out


I = SparsePauliOp.from_list([("I"*num_qubits, 1.0)])

Hs = []

for k in range(num_copies):
    n_block = observable.num_qubits

    # number of identity blocks to the "left" (higher qubits)
    left_blocks = num_copies - k - 1
    # number of identity blocks to the "right" (lower qubits)
    right_blocks = k

    left = _tensor_power(I, left_blocks)
    right = _tensor_power(I, right_blocks)

    # Build H = left ⊗ observable ⊗ right
    pieces = [p for p in (left, observable, right) if p is not None]
    H = pieces[0]
    for p in pieces[1:]:
        H = H.tensor(p)

    Hs.append(H)

In [78]:
ansatze_type = 'exact'

if potential == "QHO":
    ansatz_name = f"CQAVQE_QHO_{ansatze_type}"
elif (potential != "QHO") and (cutoff <= 64):
    ansatz_name = f"CQAVQE_{potential}{cutoff}_{ansatze_type}"
else:
    ansatz_name = f"CQAVQE_{potential}16_{ansatze_type}"

ansatz = ansatze.get(ansatz_name)
num_params = ansatz.n_params
block = ansatze.pl_to_qiskit(ansatz, num_qubits=num_qubits, reverse_bits=True)
block_params = list(block.parameters) 

# Total number of parameters = num_copies * num_params
theta = ParameterVector("θ", num_copies * num_params)

# Full circuit has num_copies * block_size qubits
full = QuantumCircuit(num_copies * num_qubits)

for k in range(num_copies):
    # parameters for copy k: theta[k*P : (k+1)*P]
    start = k * num_params
    stop = (k + 1) * num_params
    theta_block = theta[start:stop]   # length num_params

    # map template parameters -> this copy's parameters
    mapping = {p: theta_block[i] for i, p in enumerate(block_params)}

    # create the k-th copy with its own params
    block_k = block.assign_parameters(mapping, inplace=False)

    # qubits this copy should act on: contiguous block of size block_size
    qubits_k = list(range(k * num_qubits, (k + 1) * num_qubits))

    # stamp this copy into the full circuit
    full.compose(block_k, qubits=qubits_k, inplace=True)

print(full.draw("text"))
print("Full circuit parameters:", full.parameters)



                      
q_0: ─────────────────
     ┌───┐┌──────────┐
q_1: ┤ X ├┤ Ry(θ[0]) ├
     └───┘└──────────┘
q_2: ─────────────────
     ┌───┐┌──────────┐
q_3: ┤ X ├┤ Ry(θ[1]) ├
     └───┘└──────────┘
q_4: ─────────────────
     ┌───┐┌──────────┐
q_5: ┤ X ├┤ Ry(θ[2]) ├
     └───┘└──────────┘
Full circuit parameters: ParameterView([ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]), ParameterVectorElement(θ[2])])


In [84]:
# 4. Use StatevectorEstimator to get both energies -----------
estimator = StatevectorEstimator()

params = np.random.random(num_params*num_copies)
params = [1.0]*(num_params*num_copies)

# One "pub" with: (circuit, [H_A, H_B])
job = estimator.run([(full, Hs, params)])
result = job.result()

Es = result[0].data.evs
for i, E in enumerate(Es):
   print(f"Subsystem {i} energy: {E:.6f}")



Subsystem 0 energy: 0.229849
Subsystem 1 energy: 0.229849
Subsystem 2 energy: 0.229849
